In [715]:
from datetime import datetime, timedelta
import pandas as pd
import pandas
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.fftpack import fft

In [716]:
numFiles = 5
cgm_values = [pd.read_csv('dm_proj1_data/CGMSeriesLunchPat{}.csv'.format(i)) for i in range(1, numFiles + 1)]
cgm_timestamps = [pd.read_csv('dm_proj1_data/CGMDatenumLunchPat{}.csv'.format(i)) for i in range(1, numFiles + 1)]

In [717]:
def mdate_to_pdate(mdate):
    return (datetime.fromordinal(int(mdate)) + timedelta(days=mdate % 1) - timedelta(days=366)).strftime("%Y-%m-%d %H:%M:%S")

In [718]:
# print('first', mdate_to_pdate(737225.5842))
# print('last', mdate_to_pdate(737225.48))

In [719]:
def epochToDate(cgmT):
    for col in cgmT.columns:
        cgmT[col] = cgmT[col].apply(lambda x: mdate_to_pdate(x) if pd.notnull(x) else x)

for i in range(numFiles):
    epochToDate(cgm_timestamps[i])

In [720]:
for i in range(numFiles):
    cgm_timestamps[i], cgm_values[i] = cgm_timestamps[i].T, cgm_values[i].T

In [721]:
for i in range(numFiles):
    cgm_timestamps[i] = cgm_timestamps[i].set_index([pandas.Index(['cgm_{}'.format(i) for i in range(1, len(cgm_timestamps[i]) + 1)])])
    cgm_values[i] = cgm_values[i].set_index([pandas.Index(['cgm_{}'.format(i) for i in range(1, len(cgm_timestamps[i]) + 1)])])

In [722]:
cgm_merged = []
for i in range(numFiles):
    cgm_merged.append(pandas.merge(cgm_timestamps[i], cgm_values[i], how="inner", left_index=True, right_index=True))

In [723]:
def colNames(cgmM):
    cols = cgmM.columns.tolist()
    new_cols = []
    for i in range(len(cols) // 2):
        new_cols.extend([cols[i], cols[len(cols) // 2 + i]])
    return new_cols


cols = []
for i in range(numFiles):
    cols.append(colNames(cgm_merged[i]))

In [724]:
for i in range(numFiles):
    cgm_merged[i] = cgm_merged[i][cols[i]]
    cgm_merged[i] = cgm_merged[i].fillna(method='ffill', limit=2)

In [725]:
# sort timeseries
for j in range(numFiles):
    for i in cgm_merged[j].columns:
        cgm_merged[j][i] = cgm_merged[j][i].values[::-1]

In [726]:
# for i in range(1, numFiles + 1):
#     cgm_merged.to_csv("dm_proj1_data/intermediate_files/cgm_merged_{}.csv".format(i))

In [727]:
# cgmM = cgm_merged[0]
# for cgmM in cgm_merged:
#     for i in range(len(cgmM)):
#         x, y = str(i) + "_x", str(i) + "_y"
#         plt.plot(cgmM[x], cgmM[y])
#         plt.title("Patient: 1 Meal #{}".format(x.split("_")[0]))
#         plt.xticks(rotation=90)
#         plt.savefig("Meal_{}.png".format(x.split("_")[0]))
#         plt.close()


In [728]:
# list(cgm_merged[1].columns)
len(cgm_merged[1].columns)//2
# cgm_merged[1]

38

In [729]:
skipCols=[]
for idx in range(numFiles):
    delCol = []
    for i in range(len(cgm_merged[idx].columns) // 2):
        y = '{}_y'.format(i)
        delCol.append(i) if sum(pd.isnull(cgm_merged[idx][y])) >= len(cgm_merged[idx][y]) // 2 else None
    skipCols.append({} if not delCol else {j:None for j in delCol})
print(skipCols)

def isNaN(skipCols,idx,i):
    return False if not skipCols[idx] else True if skipCols[idx].__contains__(i) else False

[{}, {10: None, 16: None}, {0: None, 9: None, 19: None, 24: None, 29: None, 58: None, 72: None}, {12: None, 16: None, 20: None, 28: None}, {3: None}]


In [730]:
def addOutofRange(cgmMO, fileIdx):
    num = 2
    for i in range(len(cgmMO.columns) // 2):
        if not isNaN(skipCols, fileIdx, i):
            cgmMO.insert(num, '{}_out_of_range'.format(i), [70 - j if j < 70 else j - 180 if j > 180 else 0 for j in cgmMO['{}_y'.format(i)]], True)
            num += 3
        else:
            num+=2
def delOutOfRange(cgmMO, fileIdx):
    out = []
    for i in range(len(cgmMO.columns) // 3):
        if not isNaN(skipCols,fileIdx,i):
            out.append(round(sum(cgmMO['{}_out_of_range'.format(i)]) / len(cgmMO['{}_out_of_range'.format(i)]), 2))
            del cgmMO['{}_out_of_range'.format(i)]
    if fileIdx!=0:
        cgmMO.drop(cgmMO.columns[-1],axis=1,inplace=True)
    return out


for i in range(numFiles):
    addOutofRange(cgm_merged[i],i)

In [731]:
help(cgm_merged[0].drop)

Help on method drop in module pandas.core.frame:

drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise') method of pandas.core.frame.DataFrame instance
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by specifying directly index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or
        columns (1 or 'columns').
    index, columns : single label or list-like
        Alternative to specifying axis (``labels, axis=1``
        is equivalent to ``columns=labels``).
    
        .. versionadded:: 0.21.0
    level : int or level name, optional
        For MultiIndex, level

In [732]:
def countOutOfRange(cgmMO,fileIdx):
    posCount, negCount = 0, 0
    outOfRange = []
    for i in range(len(cgmMO.columns) // 3):
        if not isNaN(skipCols,fileIdx,i):
            for j in cgmMO['{}_out_of_range'.format(i)]:
                if j > 0:
                    posCount += 1
                if j < 0:
                    negCount += 1
        #     outOfRange.append([posCount,negCount])
            outOfRange.append(posCount)
            posCount, negCount = 0, 0
    return outOfRange


outOfRangeList = []
for i in range(numFiles):
    outOfRangeList.append([countOutOfRange(cgm_merged[i],i), delOutOfRange(cgm_merged[i],i)])

In [733]:
len(outOfRangeList[3][0])

46

In [734]:
timeRange = [
    [
        '{1} - {0}'.format(cgm_merged[idx]['{}_x'.format(i)][0], cgm_merged[idx]['{}_x'.format(i)][-1]) for i in range(len(outOfRangeList[idx][0]))
    ] for idx in range(numFiles)
]

feature_matrix = [pd.DataFrame({'timeRange': timeRange[i], '#OutOfRange': outOfRangeList[i][0], 'meanOutOfRange':outOfRangeList[i][1]}) for i in range(numFiles)]

In [735]:
for i in range(numFiles):
    print(len(feature_matrix[i]))
    print(len(outOfRangeList[i][0]))

33
33
35
35
66
66
46
46
16
16


In [736]:
def mk_test(DF, alpha=0.3):
    """
    Input:
        DF:   a vector of data
        alpha: significance level (0.05 default)

    Output:
        trend: tells the trend (increasing, decreasing or no trend)
        h: True (if trend is present) or False (if trend is absence)
        p: p value of the significance test
        z: normalized test statistics

    Examples
    --------
      >>> DF = np.random.rand(100)
      >>> trend,h,p,z = mk_test(DF,0.05)
    """

    n = len(DF)

    # calculate S
    s = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            s += np.sign(DF[j] - DF[k])

    # calculate the unique data
    unique_x = np.unique(DF)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n * (n - 1) * (2 * n + 5)) / 18
    else:  # there are some ties in data
        tp = np.zeros(unique_x.shape)
        for i in range(len(unique_x)):
            tp[i] = np.sum(DF == unique_x[i])
        var_s = (n * (n - 1) * (2 * n + 5) - np.sum(tp * (tp - 1) * (2 * tp + 5))) / 18

    if s > 0:
        z = (s - 1) / np.sqrt(var_s)
    elif s < 0:
        z = (s + 1) / np.sqrt(var_s)
    else:  # s == 0:
        z = 0

    # calculate the p_value
    p = 2 * (1 - norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1 - alpha / 2)

    trend = 'decreasing' if z < 0 and h else 'increasing' if z > 0 and h else 'no trend'

    return trend, h, p, z

In [737]:
# for j in range(numFiles):
#     for i in range(len(cgm_merged[j].columns) // 2):
#         col = str(i) + '_y'
#         print("Meal #{} :{}".format(i, mk_test(cgm_merged[j][col].values.ravel())))

In [738]:
for i in range(numFiles): print(len(feature_matrix[i]))
temp = [[j for j in range(len(cgm_merged[i].columns)//2) if not isNaN(skipCols,i,j)] for i in range(numFiles)]
# for idx in range(numFiles):
#     # Insert mk_testt_trend
#     print(len(temp[idx]))
#     feature_matrix[idx].insert(3, "mk_test_trend", [mk_test(cgm_merged[idx][str(i) + '_y'].values.ravel())[0] for i in temp[idx]])

#     # Insert mk_test_p_value: p value of the significance test
#     feature_matrix[idx].insert(4, "mk_test_p_value", [mk_test(cgm_merged[idx][str(i) + '_y'].values.ravel())[2] for i in temp[idx]])

#     # Insert mk_test_z_value: normalized test statistics
#     feature_matrix[idx].insert(5, "mk_test_z_value", [mk_test(cgm_merged[idx][str(i) + '_y'].values.ravel())[3] for i in temp[idx]])
# for i in range(numFiles): print(len(temp[i]))
# #     print(temp[idx])
# temp[0]
# idx=3
# [mk_test(cgm_merged[idx][str(i) + '_y'].values.ravel())[0] for i in temp[idx]]
# len(temp[3])
# len(outOfRangeList[3][0])
print()
for i in range(numFiles): print(cgm_merged[i].columns)
for i in range(numFiles): print(sum([1 for j in range(len(cgm_merged[i].columns)//2) if not isNaN(skipCols,i,j)]))

33
35
66
46
16

Index(['0_x', '0_y', '1_x', '1_y', '2_x', '2_y', '3_x', '3_y', '4_x', '4_y',
       '5_x', '5_y', '6_x', '6_y', '7_x', '7_y', '8_x', '8_y', '9_x', '9_y',
       '10_x', '10_y', '11_x', '11_y', '12_x', '12_y', '13_x', '13_y', '14_x',
       '14_y', '15_x', '15_y', '16_x', '16_y', '17_x', '17_y', '18_x', '18_y',
       '19_x', '19_y', '20_x', '20_y', '21_x', '21_y', '22_x', '22_y', '23_x',
       '23_y', '24_x', '24_y', '25_x', '25_y', '26_x', '26_y', '27_x', '27_y',
       '28_x', '28_y', '29_x', '29_y', '30_x', '30_y', '31_x', '31_y', '32_x',
       '32_y'],
      dtype='object')
Index(['0_x', '0_y', '1_x', '1_y', '2_x', '2_y', '3_x', '3_y', '4_x', '4_y',
       '5_x', '5_y', '6_x', '6_y', '7_x', '7_y', '8_x', '8_y', '9_x', '9_y',
       '10_x', '10_y', '11_x', '11_y', '12_x', '12_y', '13_x', '13_y', '14_x',
       '14_y', '15_x', '15_y', '16_x', '16_y', '17_x', '17_y', '18_x', '18_y',
       '19_x', '19_y', '20_x', '20_y', '21_x', '21_y', '22_x', '22_y', '23_x',
      

[                                    timeRange  #OutOfRange  meanOutOfRange
 0   2018-06-14 14:01:11 - 2018-06-14 11:31:11           14           25.94
 1   2018-06-06 15:04:00 - 2018-06-06 12:38:59           31          123.97
 2   2018-06-05 13:13:53 - 2018-06-05 10:48:54           16           35.16
 3   2018-06-04 13:43:48 - 2018-06-04 11:13:49           13           13.58
 4   2018-05-21 14:08:44 - 2018-05-21 11:43:45            0            0.00
 5   2018-05-16 14:34:42 - 2018-05-16 12:04:40           15           15.87
 6   2018-03-21 14:04:05 - 2018-03-21 11:39:04            5            0.74
 7   2018-03-01 13:44:01 - 2018-03-01 11:19:02           12           14.87
 8   2018-02-26 13:43:46 - 2018-02-26 11:13:47            6            0.84
 9   2018-02-21 13:49:44 - 2018-02-21 11:24:40            7            1.77
 10  2018-02-16 15:10:02 - 2018-02-16 12:45:01           17           21.35
 11  2018-02-15 13:34:56 - 2018-02-15 11:09:55           12           18.00
 12  2018-02

In [314]:
def varFFT(CGMO, fileIdx):
    finalList = []
    _, l = CGMO.shape
    l = int(l/2)
    for i in range(l):
        if not isNaN(skipCols, fileIdx, l):
            x, y = str(i) + "_x", str(i) + "_y"
            Gval = CGMO[y]
            yf = fft(Gval)
            finalList.append(np.var(yf))
    return finalList


varFFTList = []
for i in range(numFiles):
    varFFTList.append(varFFT(cgm_merged[i], i))

KeyError: '75_y'

In [ ]:
for i in range(numFiles):
    print(len(varFFTList[i]),len(feature_matrix[i]))


In [ ]:
for idx in range(numFiles):
    feature_matrix[idx].insert(6, "something_FFT", varFFTList[idx])

In [ ]:
def MeanRange(CGMMR):
    rangeList = []
    finalList = []
    for i in range(len(CGMMR.columns) // 2):
        maxValue = CGMMR[str(i) + "_y"].max()
        minValue = CGMMR[str(i) + "_y"].min()
        rangeList.append(maxValue - minValue)
    
    averageList = sum(rangeList) / len(rangeList)
    for i in range(len(rangeList)):
        finalList.append(round(rangeList[i] - averageList,2))
    
    return finalList


MeanRangeList = []
for i in range(numFiles):
    MeanRangeList.append(MeanRange(cgm_merged[i]))

In [ ]:
for idx in range(numFiles):
    feature_matrix[idx].insert(7, "MeanRange", MeanRangeList[idx])

In [ ]:
feature_matrix[4]